## Part 6: Multi-signal model
In reality, there are multiple Higgs boson processes which contribute to the total signal model, not only ggH. This section will explain how we can add an additional signal process (VBF) into the fit. Following this, we will add a second analysis category (Tag1), which has a higher purity of VBF events. To put this in context, the selection for Tag1 may require two jets with a large pseudorapidity separation and high invariant mass, which are typical properties of the VBF topology. By including this additional category with a different relative yield of VBF to ggH production, we are able to simultaneously constrain the rate of the two production modes.

In the SM, the VBF process has a cross section which is roughly 10 times smaller than the ggH cross section. This explains why we need to use certain features of the event to boost the purity of VBF events. The LO Feynman diagram for VBF production is shown below.

<img src="plots/part6_feynman.png" width="300"/>

### Building the models
Firstly, lets build the necessary inputs for this section the following code blocks. The code uses everything we have learnt in the previous sections:

1) Signal models (Gaussians) are built separately for each process (ggH and VBF) in each analysis category (Tag0 and Tag1). This uses separate `TTrees` for each contribution in the `mc_part6.root` file. The mean and width of the Gaussians include the effect of the parametric shape uncertainties, `nuisance_scale` and `nuisance_smear`. Each signal model is normalised according to the following equation, where $\epsilon_{ij}$ labels the fraction of process, $i$ (=ggH,VBF), landing in analysis category, $j$ (=Tag0,Tag1), and $\mathcal{L}$ is the integrated luminosity (defined in the datacard).

$$ N_{ij} = \sigma_i \cdot \mathcal{B}^{\gamma\gamma} \cdot \epsilon_{ij} \cdot \mathcal{L}$$

2) A background model is constructed for each analysis category by fitting the mass sidebands in data. The input data is stored in the `data_part6.root` file. The models are `RooMultiPdfs` which contain an exponential, a 4th-order Chebychev polynomial and a power law function. The shape parameters and normalisation terms of the background models are freely floating in the final fit.
* Have a look through the code and try to understand all parts of the model construction. When you are happy, go ahead and construct the models by running the blocks

In [ ]:
import ROOT
from IPython.display import Image

### Signal models

In [ ]:
# Signal modelling
# Open ROOT file containing trees for both ggH and VBF
f = ROOT.TFile("mc_part6.root")

# Define mass and weight variables
mass = ROOT.RooRealVar("CMS_hgg_mass", "CMS_hgg_mass", 125, 100, 180)
weight = ROOT.RooRealVar("weight","weight",0,0,1)

# Load the MC as RooDataSets, including the syst-varied TTrees
mc = {}

procs = ['ggH','VBF']
cats = ['Tag0','Tag1']

systs = ['scale','smear','JEC','photonID']

for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        t = f.Get(key)
        mc[key] = ROOT.RooDataSet(key, key, t, ROOT.RooArgSet(mass,weight), "", "weight" )
        for syst in systs:
            for direction in ['Up','Down']:
                key_syst = "%s_%s%s01Sigma"%(key,syst,direction)
                t = f.Get(key_syst)
                mc[key_syst] = ROOT.RooDataSet(key_syst, key_syst, t, ROOT.RooArgSet(mass,weight), "", "weight" )

In [ ]:
# Calculate the variations in mean/sigma for the systematic-varied MC
# Let's first build the model to fit
mean = ROOT.RooRealVar("mean", "mean", 125, 124, 126)
sigma = ROOT.RooRealVar("sigma", "sigma", 2, 1, 3)
gaus = ROOT.RooGaussian("model", "model", mass, mean, sigma)

mean_values, sigma_values = {}, {}
for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        gaus.fitTo(mc[key], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
        gaus.fitTo(mc[key], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
        mean_values[key] = [mean.getVal(), mean.getError()]
        sigma_values[key] = [sigma.getVal(), sigma.getError()]
        for syst in ['scale','smear']:
            key_up, key_down = "%s_%sUp01Sigma"%(key,syst), "%s_%sDown01Sigma"%(key,syst)
            gaus.fitTo(mc[key_up], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
            gaus.fitTo(mc[key_up], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
            mean_values[key_up] = [mean.getVal(), mean.getError()]
            sigma_values[key_up] = [sigma.getVal(), sigma.getError()]
            gaus.fitTo(mc[key_down], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
            gaus.fitTo(mc[key_down], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )
            mean_values[key_down] = [mean.getVal(), mean.getError()]
            sigma_values[key_down] = [sigma.getVal(), sigma.getError()]

# Store variations to bake into model
scale_variations, smear_variations = {}, {}
for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        # Scale
        syst = "scale"
        key_up, key_down = "%s_%sUp01Sigma"%(key,syst), "%s_%sDown01Sigma"%(key,syst)
        scale_variations[key] = 0.5*( abs(mean_values[key_up][0]/mean_values[key][0]-1) + abs(mean_values[key_down][0]/mean_values[key][0]-1) )
        # Smear
        syst = "smear"
        key_up, key_down = "%s_%sUp01Sigma"%(key,syst), "%s_%sDown01Sigma"%(key,syst)
        smear_variations[key] = 0.5*( abs(sigma_values[key_up][0]/sigma_values[key][0]-1) + abs(sigma_values[key_down][0]/sigma_values[key][0]-1) )

In [ ]:
# Now lets construct the signal models to save in the workspace
MH = ROOT.RooRealVar("MH", "MH", 125, 120, 130 )
MH.setConstant(True)

# Dicts to store the shape parameters
dMH, mean_formula, sigma, sigma_formula = {}, {}, {}, {}
models = {}

# Also build nuisance parameters for scale and smearing effects
eta = ROOT.RooRealVar("nuisance_scale", "nuisance_scale", 0, -5, 5)
eta.setConstant(True)
chi = ROOT.RooRealVar("nuisance_smear", "nuisance_smear", 0, -5, 5)
chi.setConstant(True)

for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        dMH[key] = ROOT.RooRealVar("dMH_%s"%key, "dMH_%s"%key, 0, -2, 2 )
        mean_formula[key] = ROOT.RooFormulaVar("mean_%s"%key, "mean_%s"%key, "(@0+@1)*(1+%.4f*@2)"%scale_variations[key], ROOT.RooArgList(MH,dMH[key],eta))

        sigma[key] = ROOT.RooRealVar("sigma_%s_nominal"%key, "sigma_%s_nominal"%key, 2, 1, 5)
        sigma_formula[key] = ROOT.RooFormulaVar("sigma_%s"%key, "sigma_%s"%key, "@0*(1+%.4f*@1)"%smear_variations[key], ROOT.RooArgList(sigma[key],chi))

        models[key] = ROOT.RooGaussian( "model_%s"%key, "model_%s"%key, mass, mean_formula[key], sigma_formula[key] )

        # Fit model to MC
        models[key].fitTo( mc[key], ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

        # Set shape parameters of model to be constant (i.e. fixed in fit to data)
        dMH[key].setConstant(True)
        sigma[key].setConstant(True)


In [ ]:
# Now let's build the normalisation objects

# From LHCHWG twiki: https://twiki.cern.ch/twiki/bin/view/LHCPhysics/CERNYellowReportPageAt13TeV
xs = {}
xs['ggH'] = ROOT.RooRealVar("xs_ggH", "Cross section of ggH in [pb]", 48.58 )
xs['ggH'].setConstant(True)
xs['VBF'] = ROOT.RooRealVar("xs_VBF", "Cross section of VBF in [pb]", 3.782 )
xs['VBF'].setConstant(True)

br_gamgam = ROOT.RooRealVar("BR_gamgam", "Branching ratio of Higgs to gamma gamma", 2.7e-3 )
br_gamgam.setConstant(True)

# Calculate the efficiency of selection for the different proc/cat combinations and define norm obkects
eff, norms = {}, {}
for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        sumw = mc[key].sumEntries()
        e = sumw/(xs[proc].getVal()*br_gamgam.getVal())
        eff[key] = ROOT.RooRealVar("eff_%s"%key, "Efficiency for %s events to land in %s"%(proc,cat), e )
        # Set constant
        eff[key].setConstant(True)
        # Define normalisation objects
        norms[key] = ROOT.RooProduct("model_%s_norm"%key, "Normalisation term for %s in %s"%(proc,cat), ROOT.RooArgList(xs[proc],br_gamgam,eff[key]))


In [ ]:
f_out = ROOT.TFile("workspace_sig_part6.root", "RECREATE")
w_sig = ROOT.RooWorkspace("workspace_sig","workspace_sig")
for model in models.values(): getattr(w_sig, "import")(model)
for norm in norms.values(): getattr(w_sig, "import")(norm)
w_sig.Print()
w_sig.Write()
f_out.Close()

### Background models

In [ ]:
# Background model construction
# Lets build a RooMultiPdf to fit the background distribution in each category
mass = ROOT.RooRealVar("CMS_hgg_mass", "CMS_hgg_mass", 125, 100, 180)
weight = ROOT.RooRealVar("weight","weight",0,0,1)

# Define mass ranges to be fit to find the initial background model parameter values
mass.setRange("loSB", 100, 115 )
mass.setRange("hiSB", 135, 180 )
mass.setRange("full", 100, 180 )
fit_range = "loSB,hiSB"

In [ ]:
# Define dicts to store the background model objects
alpha, poly_1, poly_2, poly_3, poly_4, pow_1 = {}, {}, {}, {}, {}, {}
pdfs = {}
index = {}
models_bkg = {}
multipdfs = {}
norms_bkg = {}

In [ ]:
# Open the file
f = ROOT.TFile("data_part6.root","r")

cats = ['Tag0','Tag1']

data = {}
for cat in cats:

    # Build a RooDataSet for the category
    t = f.Get("data_%s"%cat)
    data[cat] = ROOT.RooDataSet("data_%s"%cat, "data_%s"%cat, t, ROOT.RooArgSet(mass), "", "weight")

    alpha[cat] = ROOT.RooRealVar("alpha_%s"%cat, "alpha_%s"%cat, -0.05, -0.2, 0 )
    pdfs["%s_exp"%cat] = ROOT.RooExponential("model_bkg_exp_%s"%cat, "model_bkg_exp_%s"%cat, mass, alpha[cat] )
    pdfs["%s_exp"%cat].fitTo( data[cat], ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

    poly_1[cat] = ROOT.RooRealVar("poly_1_%s"%cat,"poly_1_%s"%cat, 0.01, -4, 4)
    poly_2[cat] = ROOT.RooRealVar("poly_2_%s"%cat,"poly_2_%s"%cat, 0.01, -4, 4)
    poly_3[cat] = ROOT.RooRealVar("poly_3_%s"%cat,"poly_3_%s"%cat, 0.01, -4, 4)
    poly_4[cat] = ROOT.RooRealVar("poly_4_%s"%cat,"poly_4_%s"%cat, 0.01, -4, 4)
    pdfs["%s_poly"%cat] = ROOT.RooChebychev("model_bkg_poly_%s"%cat, "model_bkg_poly_%s"%cat, mass, ROOT.RooArgList(poly_1[cat],poly_2[cat],poly_3[cat],poly_4[cat]) )
    pdfs["%s_poly"%cat].fitTo( data[cat], ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

    pow_1[cat] = ROOT.RooRealVar("pow_1_%s"%cat,"pow_1_%s"%cat, -3, -10, -0.0001)
    pdfs["%s_pow"%cat] = ROOT.RooGenericPdf("model_bkg_pow_%s"%cat, "TMath::Power(@0,@1)", ROOT.RooArgList(mass,pow_1[cat]) )
    pdfs["%s_pow"%cat].fitTo( data[cat], ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

    # Make RooCategory index object to control which pdf is acitve
    index[cat] = ROOT.RooCategory("pdfindex_%s"%cat, "Index of Pdf which is active for %s"%cat )

    # Make ArgList of models
    models_bkg[cat] = ROOT.RooArgList()
    models_bkg[cat].add( pdfs["%s_exp"%cat] )
    models_bkg[cat].add( pdfs["%s_poly"%cat] )
    models_bkg[cat].add( pdfs["%s_pow"%cat] )

    # Build the RooMultiPdf object
    multipdfs[cat] = ROOT.RooMultiPdf("multipdf_%s"%cat, "Multipdf for %s"%cat, index[cat], models_bkg[cat])

    # As usual for the data driven fit we want the background to have a freely floating yield
    norms_bkg[cat] = ROOT.RooRealVar("multipdf_%s_norm"%cat, "Number of background events in %s"%cat, data[cat].numEntries(), 0, 3*data[cat].numEntries() )

In [ ]:
# Lets again save the data as RooDataHist with 320 bins
mass.setBins(320)
data_hist = {}
for cat in cats:
    data_hist[cat] = ROOT.RooDataHist("data_hist_%s"%cat, "data_hist_%s"%cat, mass, data[cat])

In [ ]:
# Now save the background models and data sets to a workspace
f_out = ROOT.TFile("workspace_bkg_part6.root", "RECREATE")
w_bkg = ROOT.RooWorkspace("workspace_bkg","workspace_bkg")
for cat in cats:
    getattr(w_bkg, "import")(data_hist[cat])
    getattr(w_bkg, "import")(index[cat])
    getattr(w_bkg, "import")(norms_bkg[cat])
    getattr(w_bkg, "import")(multipdfs[cat])
w_bkg.Print()
w_bkg.Write()
f_out.Close()

### Datacards
Firstly, we need to check the yield variations from the JEC and photonID systematics to add to the datacard:

In [ ]:
print(" --> Yield variations to add to the datacard...")
for proc in procs:
    for cat in cats:
        key = "%s_%s"%(proc,cat)
        for syst in ['JEC','photonID']:
            key_up, key_down = "%s_%sUp01Sigma"%(key,syst), "%s_%sDown01Sigma"%(key,syst)
            yield_variation_up = mc[key_up].sumEntries()/mc[key].sumEntries()
            yield_variation_down = mc[key_down].sumEntries()/mc[key].sumEntries()
            print("Yield variation for %s systematic for (%s,%s): %.3f/%.3f"%(syst,proc,cat,yield_variation_down,yield_variation_up))

The datacards for the two analysis categories are saved separately as `datacard_part6_Tag0.txt` and `datacard_part6_Tag1.txt`. Lets open them up and have a look:

In [ ]:
# Let's open the datacard and take a look
with open("datacard_part6_Tag0.txt","r") as f:
    lines = f.readlines()
    
print("".join(lines))

print("\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\n")

with open("datacard_part6_Tag1.txt","r") as f:
    lines = f.readlines()
    
print("".join(lines))

* Do you understand the changes made to include multiple signal processes in the datacard? What value in the `process` line is used to label VBF as a signal?
* Try compiling the individual datacards. What are the prefit ggH and VBF yields in each analysis category? You can find these by opening the workspace and printing the contents.
* Run the best fits and plot the prefit and postfit S+B models along with the data (see code in part 2). How does the absolute number of data events in Tag1 compare to Tag0? What about the signal-to-background ratio, S/B? 

In [ ]:
# Compile the individual datacards here

In [ ]:
# Open the workspaces and print the contents. What are the ggH and VBF yields in each analysis category

In [ ]:
# Run the best fits and save the postfit snapshot

In [ ]:
# Plot the postfit S+B mgg distributions (using code from part2)

In order to combine the two categories into a single datacard, we make use of the `combineCards.py` script:

In [ ]:
%%bash
combineCards.py datacard_part6_Tag0.txt datacard_part6_Tag1.txt > datacard_part6_combined.txt

### Running the fits
If we use the default `text2workspace` command on the combined datacard, then this will introduce a single signal strength modifer which modifies the rate of all signal processes (ggH and VBF) by the same factor. 
* Try compiling the combined datacard and running a likelihood scan. Does the sensitivity to the global signal strength improve by adding the additional analysis category "Tag1"?

In [ ]:
# Compile the combined datacard into a RooWorkspace

In [ ]:
# Run the likelihood scan

In [ ]:
# Plot the scan with plot1DScan.py script: does the sensitivity improve by adding extra category?

If we want to measure the independent rates of both processes simultaneously, then we need to introduce a separate signal strength for ggH and VBF. To do this we use the `multiSignalModel` physics model in combine by adding the following options to the `text2workspace` command:

In [ ]:
%%bash
text2workspace.py datacard_part6_combined.txt -m 125 -P HiggsAnalysis.CombinedLimit.PhysicsModel:multiSignalModel \
--PO "map=.*/ggH:r_ggH[1,0,2]" --PO "map=.*/VBF:r_VBF[1,0,3]" -o datacard_part6_combined_multiSignalModel.root

The syntax for the parameter to process mapping is `map=category/process/POI[default,min,max]`. We have used the wildcard `.*` to tell combine that the POI (parameter of interest) should scale all cases of that process, regardless of the analysis category. The output of the above command tells us what is scaled by the two signal strengths. You can see this is exactly what we require!

To run a 1D "profiled" likelihood scan for ggH we use the following command:

In [ ]:
%%bash
combine -M MultiDimFit datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .scan.part6_multiSignalModel_ggH --algo grid --points 20 --cminDefaultMinimizerStrategy 0 \
--saveInactivePOI 1 -P r_ggH --floatOtherPOIs 1

* "Profiled" here means we are profiling over the other parameter of interest, `r_VBF` in the fit. In other words, we are treating `r_VBF` as an additional nuisance parameter. The option `--saveInactivePOI 1` stores the value of `r_VBF` in the combine output. Take a look at the fit output. Does the value of `r_VBF` depend on `r_ggH`? Are the two parameters of interest correlated? Remember, to look at the contents of the TTree you can use `limit->Show(i)`, where i is an integer labelling the point in the likelihood scan.
* Run the profiled scan for the VBF signal strength. Plot the `r_ggH` and `r_VBF` likelihood scans using the `plot1DScan.py` script. You will need to change some of the input options, in particular the `--POI` option.

In [ ]:
# Look at the fit output and print out the contents of the TTree

In [ ]:
%%bash
# Run the profiled likelihood scan for r_VBF

In [ ]:
%%bash
# Plot the two profiled likelihood scans. You will need to change the options --POI in plot1DScan
plot1DScan.py --help

### Two-dimensional likelihood scan
We can also run the fit at fixed points in (`r_ggH`,`r_VBF`) space. By using a sufficient number of points, we are able to up the 2D likelihood surface. Let's change the ranges of the parameters of interest to match what we have found in the profiled scans:

In [ ]:
%%bash
combine -M MultiDimFit datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .scan2D.part6_multiSignalModel --algo grid --points 800 --cminDefaultMinimizerStrategy 0 \
-P r_ggH -P r_VBF --setParameterRanges r_ggH=0.5,2.5:r_VBF=-1,2

To plot the output you can use the following code (taken from `plot_2D_scan.py`). This code interpolates the 2NLL value between the points ran in the combine scan so that the plot shows a smooth likelihood surface. You may find in some cases, the number of scanned points and interpolation parameters need to be tuned to get a sensible looking surface. This basically depends on how complicated the likelihood surface is.

In [ ]:
from scipy.interpolate import griddata
import numpy as np

ROOT.gStyle.SetOptStat(0)

# Open the combine output
f = ROOT.TFile("higgsCombine.scan2D.part6_multiSignalModel.MultiDimFit.mH125.root")
t = f.Get("limit")

# Number of points in interpolation
n_points = 1000
x_range = [0.5,2.5]
y_range = [-1,2]

# Number of bins in plot
n_bins = 40

# Load the points and NLL from the combine output
x, y, deltaNLL = [], [], []
for ev in t:
    x.append( getattr(ev,"r_ggH") )
    y.append( getattr(ev,"r_VBF") )
    deltaNLL.append( getattr(ev,"deltaNLL") )

In [ ]:
# Do interpolation
# Convert to numpy arrays as required for interpolation
dnll = np.asarray(deltaNLL)
points = np.array([x,y]).transpose()
# Set up grid
grid_x, grid_y = np.mgrid[x_range[0]:x_range[1]:n_points*1j, y_range[0]:y_range[1]:n_points*1j]
grid_vals = griddata(points,dnll,(grid_x,grid_y), "cubic")

# Remove NANS
grid_x = grid_x[grid_vals==grid_vals]
grid_y = grid_y[grid_vals==grid_vals]
grid_vals = grid_vals[grid_vals==grid_vals]

In [ ]:
# Define Profile2D histogram and fill
h2D = ROOT.TProfile2D("h","h",n_bins,x_range[0],x_range[1],n_bins,y_range[0],y_range[1])
for i in range(len(grid_vals)):
  # Factor of 2 comes from 2*NLL
  h2D.Fill( grid_x[i], grid_y[i], 2*grid_vals[i] )

# Loop over bins: if content = 0 then set 999
for ibin in range(1,h2D.GetNbinsX()+1):
  for jbin in range(1,h2D.GetNbinsY()+1):
    if h2D.GetBinContent(ibin,jbin)==0:
      xc, yc = h2D.GetXaxis().GetBinCenter(ibin), h2D.GetYaxis().GetBinCenter(jbin)
      h2D.Fill(xc,yc,999)

In [ ]:
# Set up canvas
canv = ROOT.TCanvas("canv","canv",600,600)
canv.SetTickx()
canv.SetTicky()
canv.SetLeftMargin(0.115)
canv.SetBottomMargin(0.115)
# Extract binwidth
xw = (x_range[1]-x_range[0])/n_bins
yw = (y_range[1]-y_range[0])/n_bins

# Set histogram properties
h2D.SetContour(999)
h2D.SetTitle("")
h2D.GetXaxis().SetTitle("r_ggH")
h2D.GetXaxis().SetTitleSize(0.05)
h2D.GetXaxis().SetTitleOffset(0.9)
h2D.GetXaxis().SetRangeUser(x_range[0],x_range[1]-xw)

h2D.GetYaxis().SetTitle("r_VBF")
h2D.GetYaxis().SetTitleSize(0.05)
h2D.GetYaxis().SetTitleOffset(0.9)
h2D.GetYaxis().SetRangeUser(y_range[0],y_range[1]-yw)

h2D.GetZaxis().SetTitle("-2 #Delta ln L")
h2D.GetZaxis().SetTitleSize(0.05)
h2D.GetZaxis().SetTitleOffset(0.8)

h2D.SetMaximum(25)

In [ ]:
# Make confidence interval contours
c68, c95 = h2D.Clone(), h2D.Clone()
c68.SetContour(2)
c68.SetContourLevel(1,2.3)
c68.SetLineWidth(3)
c68.SetLineColor(ROOT.kBlack)
c95.SetContour(2)
c95.SetContourLevel(1,5.99)
c95.SetLineWidth(3)
c95.SetLineStyle(2)
c95.SetLineColor(ROOT.kBlack)

# Draw histogram and contours
h2D.Draw("COLZ")

# Draw lines for SM point
vline = ROOT.TLine(1,y_range[0],1,y_range[1]-yw)
vline.SetLineColorAlpha(ROOT.kGray,0.5)
vline.Draw("Same")
hline = ROOT.TLine(x_range[0],1,x_range[1]-xw,1)
hline.SetLineColorAlpha(ROOT.kGray,0.5)
hline.Draw("Same")

# Draw contours
c68.Draw("cont3same")
c95.Draw("cont3same")

# Make best fit and sm points
gBF = ROOT.TGraph()
gBF.SetPoint(0,grid_x[np.argmin(grid_vals)],grid_y[np.argmin(grid_vals)])
gBF.SetMarkerStyle(34)
gBF.SetMarkerSize(2)
gBF.SetMarkerColor(ROOT.kBlack)
gBF.Draw("P")

gSM = ROOT.TGraph()
gSM.SetPoint(0,1,1)
gSM.SetMarkerStyle(33)
gSM.SetMarkerSize(2)
gSM.SetMarkerColor(ROOT.kRed)
gSM.Draw("P")


# Add legend
leg = ROOT.TLegend(0.6,0.67,0.8,0.87)
leg.SetBorderSize(0)
leg.SetFillColor(0)
leg.AddEntry(gBF,  "Best fit", "P" )
leg.AddEntry(c68, "1#sigma CL" , "L" )
leg.AddEntry(c95, "2#sigma CL" , "L" )
leg.AddEntry(gSM,  "SM"     , "P" )
leg.Draw()

canv.Update()
canv.Draw()

* The plot shows that the data is in agreement with the SM within the $2\sigma$ CL. Here, the $1\sigma$ and $2\sigma$ confidence interval contours corresponds to 2NLL values of 2.3 and 5.99, respectively. Do you understand why this? Think about Wilk's theorem.
* Does the plot show any correlation between the ggH and VBF signal strengths? Are the two positively or negatively correlated? Does this make sense for this pair of parameters given the analysis setup? Try repeating the 2D likelihood scan using the "Tag0" only datacard. How does the correlation behaviour change?
* How can we read off the "profiled" 1D likelihood scan constraints from this plot? 

### Correlations between parameters
For template-based analyses we can use the `FitDiagnostics` method in combine to extract the covariance matrix for the fit parameters. Unfortunately, this method is not compatible when using discrete nuisance parameters (`RooMultiPdf`). Instead, we can use the `robustHesse` method to find the Hessian matrix by finite difference methods which iteratively removes NPs until the Hessian matrix is invertable. The matrix is then inverted to get the covariance. Subsequently, we can use the covariance to extract the correlations between fit parameters. 

In [ ]:
%%bash
combine -M MultiDimFit datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .robustHesse.part6_multiSignalModel --cminDefaultMinimizerStrategy 0 -P r_ggH -P r_VBF \
--setParameterRanges r_ggH=0.5,2.5:r_VBF=-1,2 --robustHesse 1 --robustHesseSave 1 --saveFitResult

The output file `robustHesse.robustHesse.part6_multiSignalModel.root` stores the correlation matrix (`h_correlation`). This contains the correlations between all parameters including the nuisances. So if we are interested in the correlation between `r_ggH` and `r_VBF`, we first need to find which bin corresponds to these parameters:

In [ ]:
# Open combine output and load correlation matrix
f = ROOT.TFile("robustHesse.robustHesse.part6_multiSignalModel.root")
h = f.Get("h_correlation")

# Find which indices correspond to the parameters of interest (square matrix)
for i in range(1,h.GetNbinsX()+1):
    print(i, h.GetXaxis().GetBinLabel(i))

In [ ]:
# Find correlation from bin content
corr = h.GetBinContent(19,20)
print("Correlation coefficient between r_ggH and r_VBF is: %.4f"%corr)

* The two parameters of interest have a correlation coefficient of -0.198. This means the two parameters are somewhat anti-correlated. Does this match what we see in the 2D likelihood scan?

### Impacts
We extract the impacts for each parameter of interest using the following commands:

In [ ]:
%%bash
# This block may take a while... don't panic!
combineTool.py -M Impacts -d datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .impacts_part6_multiSignal --robustFit 1 --cminDefaultMinimizerStrategy 0 -P r_ggH -P r_VBF --doInitialFit

combineTool.py -M Impacts -d datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .impacts_part6_multiSignal --robustFit 1 --cminDefaultMinimizerStrategy 0 -P r_ggH -P r_VBF --doFits

combineTool.py -M Impacts -d datacard_part6_combined_multiSignalModel.root -m 125 --freezeParameters MH \
-n .impacts_part6_multiSignal --robustFit 1 --cminDefaultMinimizerStrategy 0 -P r_ggH -P r_VBF -o impacts_part6.json

plotImpacts.py -i impacts_part6.json -o impacts_part6_r_ggH --POI r_ggH
plotImpacts.py -i impacts_part6.json -o impacts_part6_r_VBF --POI r_VBF

* Examine the output PDF (and json) files. How does the impact ranking of the nuisance parameters change for the different signal strengths?

## Advanced exercises (to be added... stay tuned)
The combine experts will include additional exercises here in due course. These will include:
* Convolution of model pdfs: `RooAddPdf`
* Application of the spurious signal method
* Advanced physics models including parametrised signal strengths e.g. SMEFT
* Mass fits
* Two-dimensional parametric models 